## Importing the necessary stuff

In [ ]:
import pyautogui
from PIL import Image 
import time
from IPython.display import clear_output
import time 
import numpy as np
import matplotlib.pyplot as plt
from mss.linux import MSS as mss
import mss.tools
import cv2
from Xlib import display
from random import randint

## left_up = 0,225            ## right_up = 800, 225

## left_down = 0,560        ## right_down = 800, 560

## Capturing a screenshot and other utilities like finding the mouse position
* in `capture_resize_to_pil()`, co_ord argument takes in a tuple of the vertices, feel free to change them based on which part of the screen you want to capture
* we need to find the mouse position as well because the steering of the car is controlled by the mouse 
* `screen_cap()` is used later on because its the fastest option to capture screenshots

In [ ]:

def capture_resize_to_pil(co_ord, size = (100,100)):
    im = pyautogui.screenshot(region = co_ord ).resize((size), Image.ANTIALIAS)
    return im

def save_pil_img(img, filename):
    img.save(filename
             
def screen_cap(top = 0, left = 0, width = 800, height = 560):
    # 800x560 windowed mode
    mon = {"top": top, "left": left, "width": width, "height": height}
    sct = mss.mss()
    
    img = np.array(sct.grab(mon))
    img = cv2.resize(img, (80,25))
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    sct.close()
    return img
             
             
def find_mouse(x = False , y = False):
    
    data = display.Display().screen().root.query_pointer()._data
    res = [data["root_x"], data["root_y"]]
    
    if x == True and y == True:
        return res
    elif x == True and y == False:
        return res[0]
    else:
        return res[1]    

## Defining functions to capture "one instance" of the training data
* for each instance, a screenshot is captured on the given vertices and the x-co-ordinate of the mouse is also captured
* The mouse's x-co-ordinate is actually the label, so it is stored with the image as the filename  of the image in `collect_instance_new()`
> *what about repeating filenames due to repeating steering values while driving ?*
> * Thats taken care of by inserting a random number between 0 to 1000 after the steering value seperated by an underscore, thus reducing the probability of repeating filenames by a factor of 1000

In [ ]:

def collect_one_instance(fname_id, foldername):
    foo = capture_resize_to_pil((0,90,800,560), size = (80,56))   ## template inp
    mouse_x = find_mouse(x = True)
    fmt_name =    foldername + "/"  +str(mouse_x) + "_nu_" +  str(fname_id) +".png"
    save_pil_img(foo, fmt_name)
    print (fmt_name)
    
def collect_instance_new(fname_id, foldername):
    foo = screen_cap(top = 225,width = 800, height = 260)  ## FPV TRACK VIEW 
    mouse_x = find_mouse(x = True)
    fmt_name =    foldername + "/"  +str(mouse_x) + "_FPV_seed_"+ str(randint(0,1000))+ "__" +  str(fname_id) +".png"
    im = Image.fromarray(foo)
    im.save(fmt_name)
    
    del im
    print (fmt_name)
    
def countdown(sec):
    for i in range(sec, -1, -1):
        print(i)
        time.sleep(1)


## Data collection loop 
* the frames argument decides how many frames to collect
* also working on a simple algorithm to collect a balanced number of frames to reduce the time taken for balancing later on

In [1]:

def collect_train_data(frames):
    print (" here we go bois -- ")
    countdown(5)
    for i in range(1, frames+1):
        collect_instance_new(i, "ss")
        time.sleep(0.2)
        
        if i % 100 == 0:
            countdown(2)
            clear_output()
 
    clear_output()
            
    print("collection complete --")

In [ ]:
collect_train_data(10000)